In [6]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.2. Original Data')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Disease Status Prediction/Output')
cur_date = "011822"

library(readxl)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)

#reading in file
ozone_data = data.frame(read_excel("Ozone_Sterol_Asthma_data_011821.xlsx", sheet = 2)) %>%
    mutate(Value = ifelse(Value == "NA", NA, as.numeric(Value)))
#SHOULD MISSING DATA BE NAS OR ZEROS???

Warning message in ifelse(Value == "NA", NA, as.numeric(Value)):
“NAs introduced by coercion”


In [7]:
head(ozone_data)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,4.39
2,4,1,AS_1,AS,P,Pre,NR,NR,Lung_Function,FEV1,4.60
3,4,1,AS_1,AS,S,Post,NR,NR,Lung_Function,FEV1,4.39
4,4,1,AS_1,AS,S,Pre,NR,NR,Lung_Function,FEV1,NA
5,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,2.96
6,7,2,AS_2,AS,P,Pre,R,NR,Lung_Function,FEV1,3.08


Seeing if there are statistical differences in predictors stratified by inflammatory or lung status. These p values are relatively high, so I'm not expecting the model to predict well. 

In [15]:
#first pivoting wider to make the tbl_summary function work
wider_data = pivot_wider(ozone_data, names_from = Variable, values_from = Value)
head(wider_data)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,FEV1,⋯,Lymphocytes_Percent,Lymphocytes,Lymphocytes_Weight,BEC_Percent,BEC,BEC_Weight,SEC_Percent,SEC,Viability,NonSquamous_Percent
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,4.39,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1,AS_1,AS,P,Pre,NR,NR,Lung_Function,4.60,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1,AS_1,AS,S,Post,NR,NR,Lung_Function,4.39,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1,AS_1,AS,S,Pre,NR,NR,Lung_Function,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,2,AS_2,AS,P,Post,R,NR,Lung_Function,2.96,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,2,AS_2,AS,P,Pre,R,NR,Lung_Function,3.08,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [16]:
colnames(wider_data)

[1] "Original_Subject_No"    "Subject_No"             "Subject_ID"            
 [4] "Disease_Status"         "Sample_Type"            "Time_Point"            
 [7] "Inflammatory_Responder" "Lung_Responder"         "Category"              
[10] "FEV1"                   "FEV1_Percent"           "FEV1_Change"           
[13] "FVC1"                   "FVC1_Percent"           "FVC1_Change"           
[16] "FEV1/FVC1"              "IFNg"                   "IL10"                  
[19] "IL12p70"                "IL13"                   "IL1b"                  
[22] "IL2"                    "IL4"                    "IL6"                   
[25] "IL8"                    "TNFa"                   "GMCSF"                 
[28] "IL12p40"                "IL15"                   "IL16"                  
[31] "IL17"                   "IL1a"                   "IL5"                   
[34] "IL7"                    "TNFb"                   "VEGF"                  
[37] "Eotaxin"                "Eotaxin3"               "IP10"                  
[40] "MCP1"                   "MCP4"                   "MDC"                   
[43] "MIP1a"                  "MIP1b"                  "TARC"                  
[46] "MPO"                    "Utero"                  "25OH_VitD"             
[49] "24OH_Chol"              "SecoB"                  "25OH_Chol"             
[52] "7Keto_Chol"             "27OH_Chol"              "bEP_Chol"              
[55] "Lan"                    "Zym"                    "DHL"                   
[58] "7DHD"                   "Des"                    "dHLan"                 
[61] "Zyme"                   "Lath"                   "7DHC"                  
[64] "8DHC"                   "Chol"                   "Sample_Weight"         
[67] "TCC"                    "TCC_Weight"             "Neutrophils_Percent"   
[70] "Neutrophils"            "Neutrophils_Weight"     "Neutrophils_Change"    
[73] "Macrophages_Percent"    "Macrophages"            "Macrophages_Weight"    
[76] "Eosinophils_Percent"    "Eosinophils"            "Eosinophils_Weight"    
[79] "Lymphocytes_Percent"    "Lymphocytes"            "Lymphocytes_Weight"    
[82] "BEC_Percent"            "BEC"                    "BEC_Weight"            
[85] "SEC_Percent"            "SEC"                    "Viability"             
[88] "NonSquamous_Percent"

In [18]:
#summary statistics df
pivot_wider(ozone_data, names_from = Variable, values_from = Value) %>%
  tbl_summary(by = Inflammatory_Responder, missing = "no", 
  include = c('FEV1','FEV1_Percent','FEV1_Change','FVC1','FVC1_Percent','FVC1_Change',
              'FEV1/FVC1','IFNg','IL10','IL12p70','IL13','IL1b','IL2','IL4','IL6','IL8','TNFa','GMCSF','IL12p40',
              'IL15','IL16','IL17','IL1a','IL5','IL7','TNFb','VEGF','Eotaxin','Eotaxin3','IP10','MCP1','MCP4',
              'MDC','MIP1a','MIP1b','TARC','MPO','Utero','25OH_VitD','24OH_Chol','SecoB','25OH_Chol','7Keto_Chol',
              '27OH_Chol','bEP_Chol','Lan','Zym','DHL','7DHD','Des','dHLan','Zyme','Lath','7DHC','8DHC','Chol',
              'Sample_Weight','TCC','TCC_Weight','Neutrophils_Percent','Neutrophils','Neutrophils_Weight',
              'Neutrophils_Change','Macrophages_Percent','Macrophages','Macrophages_Weight','Eosinophils_Percent',
              'Eosinophils','Eosinophils_Weight','Lymphocytes_Percent','Lymphocytes','Lymphocytes_Weight',
              'BEC_Percent','BEC','BEC_Weight','SEC_Percent','SEC','Viability','NonSquamous_Percent'), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% #adding p value from anova
  as_tibble() 

**Characteristic**,**N**,"**NR**, N = 128","**R**, N = 256",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
FEV1,94,3.92 (0.65),3.60 (0.77),0.055
FEV1_Percent,94,100 (11),96 (16),0.2
FEV1_Change,92,-5 (6),-11 (10),0.009
FVC1,94,4.42 (0.79),4.30 (1.09),0.6
FVC1_Percent,94,97 (18),98 (15),0.7
FVC1_Change,92,-11 (8),-12 (7),0.7
FEV1/FVC1,94,89 (9),85 (9),0.022
IFNg,95,9.3 (17.8),4.5 (5.8),0.053
IL10,95,1.28 (1.87),1.43 (2.82),0.8


# K Nearest Neighbor

The goal of this analysis is to use cytokine/ sterol data [continuous] to predict inflammatory/lung staus (dichotomous). This will be done using Support Vector Machine (SVM) based on a logistic regression model for classification. It was also chosen since SVM does a better job at handling a large number of predictors, since p < n. 

# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)